In [1]:
# rules

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from datetime import datetime 
from sklearn.metrics import r2_score
import random

## Задача: предсказать средний балл на экзамене по математике

### Загрузка тренировочной выборки

In [3]:
data = pd.read_csv("data/train.csv")

In [4]:
last_index = data.shape[0] - 1

X = np.array(data.iloc[0:last_index, 2:7].copy())
y = np.array(data.iloc[0:last_index, 11].copy())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# random_state =j

In [5]:
test = pd.read_csv('data/test.csv')

In [6]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле
        
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):        
        return np.mean(self.labels)  

In [7]:
# реализуем генерацию N бутстрап-выборок

random.seed(42)

def get_bootstrap(data, labels, N):
    n_samples = data.shape[0]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        
        for j in range(n_samples):
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

In [8]:
def dispersion(labels):
        
    return np.var(labels)


def get_subsample(len_sample):
    # будем сохранять не сами признаки, а их индексы
    sample_indexes = [i for i in range(len_sample)]
    
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    
    random.shuffle(sample_indexes)
    subsample = sample_indexes[0:len_subsample].copy()
#     for _ in range(len_subsample):
#         subsample.append(sample_indexes.pop())
        
    return subsample

def mean_squared_error(y_real, prediction):
    return (sum((y_real - prediction)**2)) / len(y_real)


# Расчет качества

def quality(left_labels, right_labels, current_dispersion):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_dispersion - p * dispersion(left_labels) - (1 - p) * dispersion(right_labels)

# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    global min_leafs
    
    min_leafs = 5

    current_dispersion = dispersion(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    # выбор индекса из подвыборки длиной sqrt(n_features)
    subsample = get_subsample(n_features)
    
    for index in subsample:
        t_values = [row[index] for row in data]
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leafs or len(false_data) < min_leafs:
                continue
            
            current_quality = quality(true_labels, false_labels, current_dispersion)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):
    global level_count, leaf_count, level_limit, leaf_limit

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
#     if quality == 0 or leaf_count + 1 == leaf_limit or level_count == level_limit:    
    if quality == 0:        

        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    level_count += 1
    
    leaf_count += 1   
    true_branch = build_tree(true_data, true_labels)
    leaf_count += 1   
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [9]:
# добавим функцию формирования случайного леса.

def random_forest(data, labels, n_trees, n_levels, n_leafs, n_min_leafs):
    global level_count, leaf_count, level_limit, leaf_limit, min_leafs    
    
    level_limit = n_levels
    leaf_limit = n_leafs
    min_leafs = n_min_leafs
    
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels in bootstrap:
        level_count, leaf_count = 0, 0
        forest.append(build_tree(b_data, b_labels))
        
    return forest

In [10]:
# Функция классификации отдельного объекта

def predict_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer
    
    if obj[node.index] <= node.t:
        return predict_object(obj, node.true_branch)
    else:
        return predict_object(obj, node.false_branch)

# функция формирования предсказания по выборке на одном дереве

def predict(data, tree):
    
    predictions = []
    for obj in data:
        prediction = predict_object(obj, tree)
        predictions.append(prediction)
    return predictions

# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    answers = []
    for tree in forest:
        answers.append(predict(data, tree))
           
    return list(map(np.mean, zip(*answers)))

In [11]:
def evaluate_alg(X_train, X_test, y_train, y_test, trees, n_trees, n_levels, n_leafs, n_min_leafs, fit_seconds):
    
    print()    
    
    start_time = datetime.now()
    train_answers = tree_vote(trees, X_train) 
    predict_seconds = (datetime.now() - start_time).total_seconds()
    print(f'train\t{mean_squared_error(y_train, train_answers):.3f}\t{n_trees}\t{n_levels}\t{n_leafs}\t{n_min_leafs}\t{fit_seconds:.3f}\t{predict_seconds:.3f}\t{r2_score(y_train, train_answers)}')
    
    start_time = datetime.now()
    test_answers = tree_vote(trees, X_test)
    predict_seconds = (datetime.now() - start_time).total_seconds()
    print(f'test\t{mean_squared_error(y_test, test_answers):.3f}\t{n_trees}\t{n_levels}\t{n_leafs}\t{n_min_leafs}\t{fit_seconds:.3f}\t{predict_seconds:.3f}\t{r2_score(y_test, test_answers)}')
    
    return train_answers, test_answers

In [14]:
# Число деревьев в ансамбле
n_trees = [6, 7]

# Максимальная глубина деревьев
n_levels = [7]

n_leafs = [7, 8, 9, 10, 11]
n_min_leafs = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

print(f'\terror\ttrees\tlevels\tleafs\tobjects\tgb fit\tpredict\tr2')

for i in range(len(n_trees)):
    for j in range(len(n_levels)):
        for k in range(len(n_leafs)):
            for m in range(len(n_min_leafs)):
            
                start_time = datetime.now()
                
                my_forest = random_forest(X_train, y_train, n_trees[i], n_levels[j], n_leafs[k], n_min_leafs[m])

                seconds = (datetime.now() - start_time).total_seconds()

                train_answers, test_answers = evaluate_alg(X_train, X_test, y_train, y_test, my_forest, n_trees[i], n_levels[j], n_leafs[k], n_min_leafs[m], seconds)

	error	trees	levels	leafs	objects	gb fit	predict	r2

train	38.065	6	7	7	3	67.387	0.205	0.7879552294226327
test	42.725	6	7	7	3	67.387	0.072	0.7801517675959991

train	38.612	6	7	7	4	65.548	0.204	0.7849089353364814
test	42.879	6	7	7	4	65.548	0.072	0.7793605342619234

train	39.176	6	7	7	5	69.500	0.210	0.7817641722460851
test	43.176	6	7	7	5	69.500	0.073	0.7778331641650973

train	39.949	6	7	7	6	70.465	0.246	0.7774622804519923
test	43.875	6	7	7	6	70.465	0.079	0.7742325139220168

train	39.154	6	7	7	7	80.971	0.255	0.7818861811765595
test	44.068	6	7	7	7	80.971	0.089	0.7732438632292783

train	39.336	6	7	7	8	72.499	0.191	0.78087266340851
test	43.941	6	7	7	8	72.499	0.071	0.7738971472069339

train	41.187	6	7	7	9	61.906	0.183	0.7705638980875233
test	44.805	6	7	7	9	61.906	0.065	0.7694513940229794

train	37.585	6	7	7	10	68.888	0.205	0.7906309641621989
test	42.843	6	7	7	10	68.888	0.073	0.7795465009155419

train	41.790	6	7	7	11	63.864	0.189	0.7672014889900645
test	45.812	6	7	7	11	63.864	0.067	0.764267401


train	39.972	7	7	9	10	71.811	0.213	0.7773323662595858
test	44.383	7	7	9	10	71.811	0.075	0.77161948104504

train	39.263	7	7	9	11	71.065	0.211	0.781280731689356
test	43.181	7	7	9	11	71.065	0.076	0.7778071018494619

train	39.994	7	7	9	12	73.840	0.221	0.7772085903813297
test	44.341	7	7	9	12	73.840	0.075	0.7718345002035365

train	39.600	7	7	10	3	76.123	0.220	0.7794053346397916
test	44.127	7	7	10	3	76.123	0.076	0.7729398234232466

train	38.827	7	7	10	4	81.006	0.231	0.783710900855731
test	42.852	7	7	10	4	81.006	0.078	0.779497939832587

train	40.647	7	7	10	5	75.981	0.213	0.7735708208616846
test	45.057	7	7	10	5	75.981	0.075	0.7681522629666284

train	39.130	7	7	10	6	73.917	0.217	0.7820247637112384
test	43.089	7	7	10	6	73.917	0.079	0.7782771235982735

train	38.609	7	7	10	7	74.973	0.213	0.784922913211721
test	42.799	7	7	10	7	74.973	0.079	0.7797734253092226

train	39.697	7	7	10	8	72.871	0.217	0.7788656826783039
test	44.038	7	7	10	8	72.871	0.078	0.7733957684270037

train	41.433	7	7	10	9	74.861	0.21

train	38.755	5	5	5	3	60.728	0.171	0.7841092705905077
test	42.496	5	5	5	3	60.728	0.061	0.7813298406012135

train	37.974	5	6	8	4	55.949	0.173	0.7884641809787677
test	41.902	5	6	8	4	55.949	0.062	0.7843891022823195

train	37.350	5	6	12	2	74.643	0.197	0.7919388139430605
test	42.285	5	6	12	2	74.643	0.068	0.7824161301303024

train	38.216	6	5	5	3	69.614	0.202	0.787111825579958
test	42.267	6	5	5	3	69.614	0.072	0.7825099568950752

train	38.563	6	5	10	8	72.253	0.208	0.7851816286786323
test	42.185	6	5	10	8	72.253	0.073	0.7829322600520364

train	37.705	6	6	10	12	73.045	0.249	0.7899628985414427
test	42.156	6	6	10	12	73.045	0.082	0.7830787685440193

train	37.918	6	7	3	3	73.263	0.219	0.7887709411469433
test	41.999	6	7	3	3	73.263	0.075	0.7838864774558564

train	37.819	6	7	3	12	73.458	0.217	0.789326756870285
test	41.862	6	7	3	12	73.458	0.099	0.7845920406571958

train	38.029	6	7	5	8	72.475	0.216	0.7881535416497624
test	42.256	6	7	5	8	72.475	0.076	0.7825653841028153

train	37.586	6	7	10	2	73.680	0.218	0.7906248103361967
test	41.491	6	7	10	2	73.680	0.075	0.786503980131057

train	38.165	6	7	12	3	72.502	0.220	0.7873953015844362
test	42.161	6	7	12	3	72.502	0.086	0.7830531510964337

train	37.562	7	5	12	8	80.491	0.233	0.7907546696945639
test	42.029	7	5	12	8	80.491	0.078	0.7837312171793669

train	37.824	7	6	10	12	78.663	0.226	0.7892989594720949
test	41.778	7	6	10	12	78.663	0.079	0.7850254318315664

train	37.958	7	7	7	4	74.861	0.221	0.7885535248569819
test	41.925	7	7	7	4	74.861	0.079	0.7842679455401317

train	37.795	7	7	7	8	77.267	0.227	0.7894601750065375
test	41.971	7	7	7	8	77.267	0.078	0.7840306967018893

train	37.632	7	7	7	10	79.189	0.225	0.7903643534832787
test	41.816	7	7	7	10	79.189	0.082	0.7848317074613257

train	37.591	7	7	8	3	82.561	0.249	0.7905962382976808
test	41.482	7	7	8	3	82.561	0.085	0.7865465153977361

train	38.692	7	7	11	9	76.069	0.212	0.7844646308089193
test	42.280	7	7	11	9	76.069	0.078	0.7824439081644501

train	37.358	7	7	11	10	76.087	0.222	0.7918934883930879
test	41.764	7	7	11	10	76.087	0.078	0.7850948036464684

train	38.016	7	7	11	11	78.544	0.226	0.7882304058591314
test	42.415	7	7	11	11	78.544	0.078	0.7817496946072339

train	38.032	7	7	11	12	77.390	0.237	0.7881379873016081
test	42.454	7	7	11	12	77.390	0.081	0.7815484274251584

train	38.042	8	5	8	8	97.882	0.258	0.7880827223052628
test	42.037	8	5	8	8	97.882	0.088	0.7836923146120783

train	38.115	8	5	12	4	88.804	0.248	0.7876739486977771
test	42.154	8	5	12	4	88.804	0.084	0.7830887874841258

train	37.713	8	6	5	8	87.366	0.249	0.7899139749088061
test	41.948	8	6	5	8	87.366	0.087	0.7841487173901498

train	38.107	8	6	10	8	87.848	0.262	0.7877202589277487
test	41.745	8	6	10	8	87.848	0.090	0.7851964738336389

train	38.601	8	6	10	12	87.040	0.241	0.7849710497381976
test	42.329	8	6	10	12	87.040	0.085	0.7821906093207406

train	38.141	8	7	8	8	84.110	0.236	0.7875301528538143
test	42.143	8	7	8	8	84.110	0.083	0.7831464707767853

train	37.386	8	7	10	12	87.334	0.257	0.7917369495154515
test	42.048	8	7	10	12	87.334	0.088	0.7836373158338289

train	39.717	6	6	5	10	78.310	0.241	0.7854425391624676
test	40.318	6	6	5	10	78.310	0.086	0.7730302594953946

train	38.771	7	6	20	10	88.963	0.253	0.7905533794911944
test	40.299	7	6	20	10	88.963	0.082	0.7731346426637182

train	38.664	6	6	5	3	76.620	0.238	0.7911298076077291
test	40.216	6	6	5	3	76.620	0.079	0.7736053486742502

train	39.166	6	5	3	10	80.405	0.306	0.7884196244555843
test	40.202	6	5	3	10	80.405	0.092	0.7736835298352294

train	38.616	7	4	5	5	93.269	0.270	0.791392368174071
test	40.173	7	4	5	5	93.269	0.093	0.7738435154822841

train	38.839	6	5	20	10	75.292	0.247	0.7901888886069701
test	40.131	6	5	20	10	75.292	0.078	0.7740825661333858

train	39.153	7	5	15	10	85.260	0.261	0.7884912397592256
test	40.107	7	5	15	10	85.260	0.086	0.7742148319441543

train	38.791	7	5	3	3	85.727	0.236	0.7904483570468385  0.7812863848119824
test	40.020	7	5	3	3	85.727	0.082	0.7747053796836463  0.7735122281874947

train	38.466	7	5	20	10	86.130	0.241	0.7922026446148294
test	39.941	7	5	20	10	86.130	0.082	0.7751534358239406

train	38.399	5	6	20	10	62.287	0.191	0.792564542014591
test	39.937	5	6	20	10	62.287	0.069	0.7751772750669302

train	39.190	5	4	10	10	56.014	0.183	0.7882913892092586
test	39.877	5	4	10	10	56.014	0.064	0.7755138296941021

train	39.251	5	5	5	3	61.750	0.178	0.7879583436072085  0.7841092705905077
test	39.942	5	5	5	3	61.750	0.063	0.7751489459300218  0.7813298406012135

train	38.518	7	5	5	3	86.430	0.257	0.7919199781840904  0.7842625817272906
test	39.674	7	5	5	3	86.430	0.087	0.776652711129673   0.7793494357709444

train	37.892	7	6	3	10	86.922	0.257	0.7953039894373466
test	39.442	7	6	3	10	86.922	0.085	0.7779604647073445

train	37.782	7	5	10	3	92.573	0.256	0.7958983792717709  0.7692613585754474
test	39.267	7	5	10	3	92.573	0.087	0.7789475954759382  0.7661805196713249

In [13]:
n_trees = 7
n_levels = 7
n_leafs = 8
n_min_leafs = 3

my_forest = random_forest(X, y, n_trees, n_levels, n_leafs, n_min_leafs)

In [49]:
test_copy = np.array(test.copy())

In [50]:
answers = tree_vote(my_forest, test_copy)

In [51]:
df = pd.DataFrame({'Id' : [i for i in range(10000, 10000 + len(answers))], 'mean_exam_points' : [round(f, 1) for f in answers]})

In [52]:
df

,Id,mean_exam_points
0,10000,72.1
1,10001,72.1
2,10002,57.9
3,10003,72.1
4,10004,72.1
...,...,...
9995,19995,57.9
9996,19996,62.7
9997,19997,72.1
9998,19998,57.9


In [48]:
df.to_csv(f"data/submission_{n_trees}_{n_levels}_{n_leafs}_{n_min_leafs}.csv", index=False)

In [53]:
start_time = datetime.now()
seconds = (datetime.now() - start_time).total_seconds()
train_answers, test_answers = evaluate_alg(X_train, X_test, y_train, y_test, my_forest, 7, 7, 8, 3, seconds)


train	39.966	7	7	8	3	0.000	0.225	0.7823308032306585
test	39.445	7	7	8	3	0.000	0.075	0.7833987256201416


In [54]:
pd.DataFrame({'1' : y_test, '2' : test_answers})

,1,2
0,71.0,67.593793
1,52.0,55.076475
2,48.0,56.500944
3,39.0,46.177326
4,91.0,88.680177
...,...,...
2495,39.0,46.177326
2496,58.0,55.076475
2497,60.0,55.635001
2498,62.0,63.804791
